# Optimización del Sistema Energético Híbrido

Este cuaderno se dedica a la formulación y resolución del problema de optimización del sistema energético híbrido, utilizando Pyomo o PuLP.

In [ ]:
# Importar librerías necesarias
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Cargar datos procesados
data = pd.read_csv('../data/processed/energy_data.csv')

# Definir parámetros del modelo
cost_solar = 1000  # Costo por kW de panel solar
cost_wind = 1500   # Costo por kW de turbina eólica
cost_battery = 500  # Costo por kWh de batería
demand = data['demand'].values  # Demanda energética
emissions_limit = 1000  # Límite de emisiones de CO2

# Crear modelo de optimización
model = ConcreteModel()

# Definir variables de decisión
model.solar = Var(within=NonNegativeReals)  # kW de paneles solares
model.wind = Var(within=NonNegativeReals)   # kW de turbinas eólicas
model.battery = Var(within=NonNegativeReals)  # kWh de baterías

# Definir función objetivo
def objective_rule(model):
    return (cost_solar * model.solar + cost_wind * model.wind + cost_battery * model.battery)
model.objective = Objective(rule=objective_rule, sense=minimize)

# Definir restricciones
def demand_constraint(model, t):
    return (model.solar + model.wind + model.battery) >= demand[t]
model.demand_constraint = Constraint(range(len(demand)), rule=demand_constraint)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(model)

# Mostrar resultados
model.display()